<a href="https://colab.research.google.com/github/milton-miranda/Classificacao-desbalanceada/blob/main/Case1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Função para gerar dataset desbalanceado
gerar_dados_desbalanceados <- function(n_observacoes = 1000, desbalanceamento = 0.005) {
  # Definir a proporção da classe minoritária
  proporcao_minoritario <- desbalanceamento
  proporcao_majoritario <- 1 - proporcao_minoritario

  # Calcular o número de observações para cada classe
  n_classe_2 <- round(n_observacoes * proporcao_minoritario)
  n_classe_1 <- n_observacoes - n_classe_2

  # Criar o dataset
  dados <- data.frame(
    ID = 1:n_observacoes,
    Classe = c(rep(1, n_classe_1), rep(2, n_classe_2)),
    X1 = rnorm(n_observacoes),   # Variável aleatória 1
    X2 = rnorm(n_observacoes)    # Variável aleatória 2
  )

  # Embaralhar os dados
  dados <- dados[sample(1:n_observacoes), ]

  return(dados)
}


In [ ]:
# Gerar dataset com 10000 observações e desbalanceamento de 0,5%
dados_0_5 <- gerar_dados_desbalanceados(n_observacoes = 10000, desbalanceamento = 0.005)

# Gerar dataset com desbalanceamento de 1%
dados_1 <- gerar_dados_desbalanceados(n_observacoes = 10000, desbalanceamento = 0.01)

# Gerar dataset com desbalanceamento de 3,5%
dados_3_5 <- gerar_dados_desbalanceados(n_observacoes = 10000, desbalanceamento = 0.035)

In [ ]:
# Verificar a proporção das classes
prop.table(table(dados_0_5$Classe))
prop.table(table(dados_1$Classe))
prop.table(table(dados_3_5$Classe))


    1     2 
0.995 0.005 


   1    2 
0.99 0.01 


    1     2 
0.965 0.035 

# Aplicação de modelos

In [ ]:
## Carregar bibliotecas necessárias
library(tidyverse)
library(caret)
library(e1071)
library(randomForest)
library(pROC)
library(ROCR)

## Função para gerar dados desbalanceados
gerar_dados_desbalanceados <- function(n_observacoes = 10000, desbalanceamento = 0.005) {
  n_classe_2 <- round(n_observacoes * desbalanceamento)
  n_classe_1 <- n_observacoes - n_classe_2

  dados <- data.frame(
    Feature1 = rnorm(n_observacoes),
    Feature2 = rnorm(n_observacoes),
    Classe = factor(c(rep(1, n_classe_1), rep(2, n_classe_2)))
  )

  dados <- dados[sample(nrow(dados)), ]  # Embaralhar dados
  return(dados)
}

## Cenários de desbalanceamento
cenarios <- list(
  "0.5%" = 0.005,
  "1%" = 0.01,
  "3.5%" = 0.035
)

## Modelos a serem avaliados
modelos <- list(
  "Regressão Logística" = glm,
  "SVM (Kernel Radial)" = function(formula, data) svm(formula, data, kernel = "radial", probability = TRUE),
  "Random Forest" = randomForest
)

## Avaliação de métricas de classificação
evaluar_modelo <- function(modelo, dados_treino, dados_teste) {
  previsoes <- predict(modelo, newdata = dados_teste, type = "response")
  if (class(modelo)[1] == "svm") {
    previsoes <- as.numeric(predict(modelo, newdata = dados_teste, probability = TRUE))
  } else if (class(modelo)[1] == "randomForest") {
    previsoes <- predict(modelo, newdata = dados_teste, type = "prob")[, 2]
  }

  matriz_confusao <- confusionMatrix(as.factor(ifelse(previsoes > 0.5, 2, 1)), dados_teste$Classe, positive = "2")
  auc <- roc(dados_teste$Classe, previsoes)$auc

  list(
    Matriz_Confusao = matriz_confusao,
    AUC = auc,
    Métricas = matriz_confusao$byClass
  )
}

## Loop para rodar o estudo em cada cenário e modelo
resultados <- list()
for (cenario in names(cenarios)) {
  cat("\nRodando cenário de desbalanceamento:", cenario, "\n")
  dados <- gerar_dados_desbalanceados(10000, cenarios[[cenario]])

  set.seed(42)
  indices <- createDataPartition(dados$Classe, p = 0.7, list = FALSE)
  dados_treino <- dados[indices, ]
  dados_teste <- dados[-indices, ]

  for (modelo_nome in names(modelos)) {
    cat("\nTreinando modelo:", modelo_nome, "\n")
    modelo <- modelos[[modelo_nome]](Classe ~ ., data = dados_treino)
    resultados[[paste(cenario, modelo_nome, sep = " - ")]] <- evaluar_modelo(modelo, dados_treino, dados_teste)
  }
}

## Exibir resultados
for (resultado in names(resultados)) {
  cat("\n---", resultado, "---\n")
  print(resultados[[resultado]]$Matriz_Confusao)
  cat("AUC:", resultados[[resultado]]$AUC, "\n")
  print(resultados[[resultado]]$Métricas)
}


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Loading required package: lattice


Attaching package: ‘caret’


The following object is masked from ‘package:purrr’:

    lift


randomForest 4.7-1.2

Type rfNews() to see new features/changes/bug fixes.


Attaching package: ‘randomForest’


The following object is masked from ‘package:dplyr’:

    combine


The following object is masked from ‘package:ggplot2’:

    margin


Type 'citation("pROC")' for a citation.


Attaching package: ‘pROC’


The following objects are 


Rodando cenário de desbalanceamento: 0.5% 

Treinando modelo: Regressão Logística 


Warning message in Ops.factor(y, mu):
“‘-’ not meaningful for factors”
Warning message in Ops.factor(eta, offset):
“‘-’ not meaningful for factors”
Warning message in Ops.factor(y, mu):
“‘-’ not meaningful for factors”


ERROR: Error in glm.fit(x = structure(c(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, : NA/NaN/Inf in 'y'


Estudo 1

In [ ]:
## Carregar bibliotecas necessárias
library(tidyverse)
library(caret)
library(glmnet)
library(randomForest)
library(xgboost)
library(pROC)
library(ROCR)
library(tictoc)

## Função para gerar dados desbalanceados
criar_dados <- function(n_observacoes = 1000, proporcao_minoria = 0.005) {
  n_classe_A <- round(n_observacoes * proporcao_minoria)
  n_classe_B <- n_observacoes - n_classe_A

  dados <- data.frame(
    Classe = factor(c(rep("A", n_classe_A), rep("B", n_classe_B))),
    replicate(200, sample(c("X", "Y", "Z"), n_observacoes, replace = TRUE))
  )
  dados <- dados[sample(nrow(dados)), ]  # Embaralhar dados
  return(dados)
}

## Cenários de desbalanceamento
cenarios <- list(
  "0.5%" = 0.005,
  "1%" = 0.01,
  "3.5%" = 0.035
)

## Loop para criar dados, dividir em treino/teste, e aplicar os modelos
resultados <- list()

for (cenario in names(cenarios)) {
  cat("\nRodando cenário de desbalanceamento:", cenario, "\n")
  dados <- criar_dados(1000, cenarios[[cenario]])

  # Dividir em treino e teste
  set.seed(42)
  indices <- createDataPartition(dados$Classe, p = 0.7, list = FALSE)
  treino <- dados[indices, ]
  teste <- dados[-indices, ]

  # Oversampling da classe minoritária
  treino_balanceado <- treino %>%
    filter(Classe == "A") %>%
    slice_sample(n = sum(treino$Classe == "B")) %>%
    bind_rows(filter(treino, Classe == "B"))

  ## Regressão Logística LASSO
  cat("Treinando modelo: Regressão Logística LASSO\n")
  tic()
  x_treino <- model.matrix(Classe ~ . - 1, data = treino_balanceado)
  y_treino <- treino_balanceado$Classe
  lasso_model <- cv.glmnet(x_treino, y_treino, family = "binomial", alpha = 1)
  toc()

  x_teste <- model.matrix(Classe ~ . - 1, data = teste)
  previsoes_lasso <- predict(lasso_model, newx = x_teste, type = "response")

  ## Random Forest
  cat("Treinando modelo: Random Forest\n")
  tic()
  rf_model <- randomForest(Classe ~ ., data = treino_balanceado)
  toc()
  previsoes_rf <- predict(rf_model, newdata = teste, type = "prob")[, 2]

  ## XGBoost
  cat("Treinando modelo: XGBoost\n")
  tic()
  xgb_model <- xgboost(data = as.matrix(x_treino), label = as.numeric(y_treino) - 1,
                       nrounds = 100, objective = "binary:logistic", verbose = 0)
  toc()
  previsoes_xgb <- predict(xgb_model, newdata = as.matrix(x_teste))

  ## Avaliação dos modelos
  for (modelo in list("LASSO" = previsoes_lasso, "Random Forest" = previsoes_rf, "XGBoost" = previsoes_xgb)) {
    nome_modelo <- names(modelo)
    previsoes <- modelo[[1]]
    matriz_confusao <- confusionMatrix(as.factor(ifelse(previsoes > 0.5, "A", "B")), teste$Classe, positive = "A")
    auc <- roc(teste$Classe, previsoes)$auc

    resultados[[paste(cenario, nome_modelo, sep = " - ")]] <- list(
      Matriz_Confusao = matriz_confusao,
      AUC = auc,
      Métricas = matriz_confusao$byClass
    )
  }
}

## Exibir resultados
for (resultado in names(resultados)) {
  cat("\n---", resultado, "---\n")
  print(resultados[[resultado]]$Matriz_Confusao)
  cat("AUC:", resultados[[resultado]]$AUC, "\n")
  print(resultados[[resultado]]$Métricas)
}

## Plotar Curvas ROC
plot_roc <- function(dados_teste, previsoes, titulo) {
  roc_obj <- roc(dados_teste$Classe, previsoes)
  plot(roc_obj, main = titulo, col = "blue", lwd = 2)
  cat("AUC:", auc(roc_obj), "\n")
}

for (cenario in names(cenarios)) {
  dados <- criar_dados(1000, cenarios[[cenario]])
  set.seed(42)
  indices <- createDataPartition(dados$Classe, p = 0.7, list = FALSE)
  dados_teste <- dados[-indices, ]

  # Regressão Logística LASSO
  modelo_log <- cv.glmnet(model.matrix(Classe ~ . - 1, data = dados[indices, ]),
                          dados[indices, ]$Classe, family = "binomial", alpha = 1)
  previsoes_log <- predict(modelo_log, newx = model.matrix(Classe ~ . - 1, data = dados_teste), type = "response")
  plot_roc(dados_teste, previsoes_log, paste("Curva ROC - Regressão Logística LASSO -", cenario))

  # Random Forest
  modelo_rf <- randomForest(Classe ~ ., data = dados[indices, ])
  previsoes_rf <- predict(modelo_rf, newdata = dados_teste, type = "prob")[, 2]
  plot_roc(dados_teste, previsoes_rf, paste("Curva ROC - Random Forest -", cenario))

  # XGBoost
  modelo_xgb <- xgboost(data = as.matrix(model.matrix(Classe ~ . - 1, data = dados[indices, ])),
                        label = as.numeric(dados[indices, ]$Classe) - 1, nrounds = 100,
                        objective = "binary:logistic", verbose = 0)
  previsoes_xgb <- predict(modelo_xgb, newdata = as.matrix(model.matrix(Classe ~ . - 1, data = dados_teste)))
  plot_roc(dados_teste, previsoes_xgb, paste("Curva ROC - XGBoost -", cenario))
}

